In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DecimalType
from functools import reduce
import pyspark
import pyspark.sql.functions as f
import datetime

packages = ','.join([
    'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1',
])

spark = SparkSession \
        .builder \
        .appName("projeto-final-pmd-pedro-jean") \
        .config("spark.mongodb.input.uri","mongodb://mongo:27017/PMD2023.Mensagens") \
        .config("spark.mongodb.output.uri","mongodb://mongo:27017/PMD2023.Mensagens") \
        .config('spark.jars.packages', packages) \
        .getOrCreate()

In [ ]:
dateUDF = f.udf(lambda date, days: (date + datetime.timedelta(days=days-1)).strftime("%Y-%m-%d"))

In [ ]:
daysList = [1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 16, 17, 18, 19, 20, 21, 22, 23]
dfList = []

initDate = datetime.datetime(2022, 4, 12)

for i in daysList:
    df = spark.read.options(header='True', inferSchema='True', quote="\"", escape="\"").csv(f'./work/Mongo/day{i}.csv')
    
    treated = df.withColumn('currency', f.when((df.money.isNotNull()) & (df.money != '0'), f.regexp_replace('money', r'(\D*)(\d+(\.|,)?\d*)', '$1')).otherwise(None)) \
        .withColumn('money',  f.regexp_replace('money', r'(\D*)(\d+(\.|,)?\d*)', '$2').cast(DecimalType(20,2))) \
        .withColumn('donated', (f.col('money') != '0')) \
        .withColumn('date', f.to_date(dateUDF(f.lit(initDate), f.lit(i))))
    
    dfList.append(treated)
    

final_sdf = reduce(pyspark.sql.dataframe.DataFrame.unionByName, dfList)

final_sdf.show()

In [ ]:
final_sdf.write.format('com.mongodb.spark.sql.DefaultSource').mode("overwrite").save()